# Machine learning - Predict MatchDay

## 0. Imports

In [32]:
import pandas as pd
import math
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
import os

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## 1. Filter & Format Barcelona Dataset

### 1.1 Read in csv-files 

In [33]:
#Read in CSV's in folder
totalFileList = os.listdir(
        "/home/woutd/Documents/School/Industriele Ingenieurswetenschappen/Ma1/Sem1/Machine Learning/Opdracht/Workspace/2-Twitter/")
listFileNames = []
for fileName in totalFileList:
    if ('Barcelona' in fileName):
        listFileNames.append(fileName)
print(len(listFileNames))      
datasets = []
for fileName in  listFileNames:
    datasets.append(pd.read_csv("/home/woutd/Documents/School/Industriele Ingenieurswetenschappen/Ma1/Sem1/Machine Learning/Opdracht/Workspace/2-Twitter/" + str(fileName)))


7


### 1.2 Merge datasets into 1

In [34]:
merged_dataset = pd.DataFrame()
for dataset in datasets:
    merged_dataset = pd.concat([merged_dataset, dataset])
merged_dataset.drop_duplicates()

print(list(merged_dataset.columns))

['timestamp', 'tweet_text', 'username', 'all_hashtags', 'followers_count']


### 1.3 Format dataset [date, count]

In [35]:
def extractDate(value):
    date = pd.to_datetime(value)
    return date.date()

def incr(value):
    return value + 1

In [36]:
merged_dataset['dates'] = merged_dataset['timestamp'].apply(extractDate)
merged_dataset.drop_duplicates()

dataset = pd.DataFrame(columns=['Date', 'Count'])
for date in merged_dataset['dates']:
    if(date in dataset['Date'].unique()):
        dataset.loc[dataset['Date'] == date, 'Count'] += 1
    else:
        # print('Fail')
        dataset = pd.concat([dataset, pd.DataFrame([[date, 0]], columns=['Date', 'Count'])])
# dataset.drop_duplicates()
dataset.sort_values(by=["Date"],ascending=False, inplace=True)
print(dataset)

         Date Count
0  2022-11-17   357
0  2022-11-16   965
0  2022-11-15   892
0  2022-11-14  1243
0  2022-11-13   638
0  2022-11-12  1468
0  2022-11-11  1593
0  2022-11-10   477
0  2022-11-09  2684
0  2022-11-08  1881
0  2022-11-07  1080
0  2022-11-06  2357
0  2022-11-03   691
0  2022-11-02  1307


### 1.4 Controle data

In [38]:
dataset['MatchDay'] = False
matchdays = [pd.Timestamp(day=6, year=2022, month=11),
            pd.Timestamp(day=9, year=2022, month=11),
            pd.Timestamp(day=1, year=2022, month=11),
            pd.Timestamp(day=20, year=2022, month=11), 
            pd.Timestamp(day=27, year=2022, month=11),]
for date in dataset['Date']:
    if(pd.Timestamp(date) in matchdays):
        dataset.loc[dataset['Date'] == date, 'MatchDay'] = True
print(dataset)


         Date Count  MatchDay
0  2022-11-17   357     False
0  2022-11-16   965     False
0  2022-11-15   892     False
0  2022-11-14  1243     False
0  2022-11-13   638     False
0  2022-11-12  1468     False
0  2022-11-11  1593     False
0  2022-11-10   477     False
0  2022-11-09  2684      True
0  2022-11-08  1881     False
0  2022-11-07  1080     False
0  2022-11-06  2357      True
0  2022-11-03   691     False
0  2022-11-02  1307     False
